# CS246 - Colab 4
## Collaborative Filtering

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [7]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [8]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [10]:
# YOUR CODE HERE
print( "Number of ratings: ", training.count() )
print( "Number of movies: ", items.count() )

Number of ratings:  80000
Number of movies:  1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [11]:
# YOUR CODE HERE
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

Now compute the RMSE on the test dataset.


In [12]:
# YOUR CODE HERE
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1198740456931535


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [13]:
# YOUR CODE HERE
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

ratings = training.union(test)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

In [16]:
userRecs.printSchema()

root
 |-- user_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- item_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [27]:
movieRecs.printSchema()

root
 |-- item_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [26]:
userRecs.show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    471|[[854, 13.871313]...|
|    463|[[1512, 5.814986]...|
|    833|[[793, 5.839633],...|
|    496|[[320, 7.5215015]...|
|    148|[[1589, 11.506889...|
|    540|[[1368, 6.6163187...|
|    392|[[1368, 7.4603624...|
|    243|[[854, 6.5456104]...|
|    623|[[1084, 6.7925615...|
|    737|[[904, 9.208461],...|
|    897|[[1589, 7.4059277...|
|    858|[[1286, 10.04098]...|
|     31|[[1159, 8.880683]...|
|    516|[[1160, 10.422813...|
|    580|[[1218, 11.834221...|
|    251|[[1589, 7.23324],...|
|    451|[[1001, 8.323375]...|
|     85|[[1368, 5.6387243...|
|    137|[[1184, 12.252513...|
|    808|[[1113, 8.734197]...|
+-------+--------------------+
only showing top 20 rows



In [36]:
flat_userRecs = userRecs.withColumn("recommendations", explode(col("recommendations"))).select('user_id', 'recommendations.*')
flat_userRecs.show()

+-------+-------+---------+
|user_id|item_id|   rating|
+-------+-------+---------+
|    471|    854|13.871313|
|    471|    974|12.808196|
|    471|   1410| 12.44127|
|    471|    962|12.064404|
|    471|    454|11.722488|
|    471|    557|11.702987|
|    471|    700|11.526935|
|    471|    536|11.284226|
|    471|    807|  11.1845|
|    471|   1439|11.090632|
|    463|   1512| 5.814986|
|    463|    206|5.7889395|
|    463|    793| 5.643863|
|    463|     20|5.2952423|
|    463|   1167| 5.239401|
|    463|   1069|5.1770616|
|    463|    954| 5.071599|
|    463|   1368|5.0244617|
|    463|    745|  5.01455|
|    463|   1187|4.9975567|
+-------+-------+---------+
only showing top 20 rows



In [38]:
flat_userRecs.join(items, flat_userRecs.item_id == items.item_id).show()

+-------+-------+---------+-------+--------------------+
|user_id|item_id|   rating|item_id|               movie|
+-------+-------+---------+-------+--------------------+
|    471|    854|13.871313|    854|    Bad Taste (1987)|
|    471|    974|12.808196|    974|Eye for an Eye (1...|
|    471|   1410| 12.44127|   1410|       Harlem (1993)|
|    471|    962|12.064404|    962|Ruby in Paradise ...|
|    471|    454|11.722488|    454|Bastard Out of Ca...|
|    471|    557|11.702987|    557|Farinelli: il cas...|
|    471|    700|11.526935|    700|Miami Rhapsody (1...|
|    471|    536|11.284226|    536|      Ponette (1996)|
|    471|    807|  11.1845|    807|Poetic Justice (1...|
|    471|   1439|11.090632|   1439|Jason's Lyric (1994)|
|    463|   1512| 5.814986|   1512|World of Apu, The...|
|    463|    206|5.7889395|    206|        Akira (1988)|
|    463|    793| 5.643863|    793|     Crooklyn (1994)|
|    463|     20|5.2952423|     20|Angels and Insect...|
|    463|   1167| 5.239401|   1

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!